In [1]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)
sql_query = "SELECT * FROM coord"

# Execute the SQL query
cursor = mydb.cursor()
cursor.execute(sql_query)

rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=column_names)
cursor.close()
mydb.close()


In [2]:
import requests
import json
import mysql.connector

def create_pairs(data):
    pairs = []
    first_lat_lng = f"{data.iloc[0]['longitude']},{data.iloc[0]['latitude']}"
    pairs.append(first_lat_lng)
    for i in range(9, len(data), 10):  # Starting from index 9, select every 10th latitude and longitude
        lat_lng = f"{data.iloc[i]['longitude']},{data.iloc[i]['latitude']}"
        pairs.append(lat_lng)
    last_lat_lng = f"{data.iloc[-1]['longitude']},{data.iloc[-1]['latitude']}"
    pairs.append(last_lat_lng)
    return pairs

def send_osrm_request(pairs):
    radiuses = ['49'] * len(pairs)
    url = f"http://localhost:5000/match/v1/bicycle/{';'.join(pairs)}?overview=full&radiuses={';'.join(radiuses)}&geometries=geojson&annotations=true"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print("Error:", response.status_code)
        return None

matched_coordinates = []
batch_data = df[df['trip_id'] == 5]

pairs = create_pairs(batch_data)
matched_coordinates_response = send_osrm_request(pairs)

print("Response from OSRM API:", matched_coordinates_response)


Response from OSRM API: {'code': 'Ok', 'matchings': [{'confidence': 0.906154, 'geometry': {'coordinates': [[-84.364281, 33.77768], [-84.364234, 33.777681], [-84.364234, 33.777609], [-84.364234, 33.777681], [-84.364281, 33.77768], [-84.364281, 33.77768], [-84.364282, 33.777393], [-84.364283, 33.777279], [-84.364283, 33.777279], [-84.364284, 33.777171], [-84.364284, 33.77701], [-84.364285, 33.776874], [-84.364286, 33.776746], [-84.364308, 33.776745], [-84.364464, 33.776744], [-84.364568, 33.776743], [-84.364568, 33.776743], [-84.364574, 33.776689], [-84.364742, 33.776687], [-84.365276, 33.776677], [-84.365289, 33.776677], [-84.365347, 33.776676], [-84.36515, 33.77587], [-84.364954, 33.775072], [-84.364863, 33.7747], [-84.364783, 33.774248], [-84.364696, 33.773752], [-84.364634, 33.773507], [-84.36462, 33.773451], [-84.364561, 33.773218], [-84.364412, 33.772632], [-84.364382, 33.772521], [-84.364281, 33.772143], [-84.364277, 33.772126], [-84.364229, 33.771991], [-84.364113, 33.77173], [-8

In [18]:

import mysql.connector

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)

# Create the "osrmresponse" table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS osrmresp (
    id INT AUTO_INCREMENT PRIMARY KEY,
    response LONGTEXT
)
"""
cursor = mydb.cursor()
cursor.execute(create_table_query)
json_response = json.dumps(matched_coordinates_response)

# Insert data from matched_coordinates_response into the "osrmresponse" table
insert_query = "INSERT INTO osrmresp (response) VALUES (%s)"
cursor.execute(insert_query, (json_response,))

# Commit changes and close the cursor and MySQL connection
mydb.commit()
cursor.close()
mydb.close()


In [16]:
import mysql.connector

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)

with mydb.cursor() as cursor:
    cursor.execute("DROP TABLE IF EXISTS osrmresp")

# Commit changes and close the MySQL connection
mydb.commit()
mydb.close()

In [19]:
print("Response from OSRM API:", json_response)

Response from OSRM API: {"code": "Ok", "matchings": [{"confidence": 0.906154, "geometry": {"coordinates": [[-84.364281, 33.77768], [-84.364234, 33.777681], [-84.364234, 33.777609], [-84.364234, 33.777681], [-84.364281, 33.77768], [-84.364281, 33.77768], [-84.364282, 33.777393], [-84.364283, 33.777279], [-84.364283, 33.777279], [-84.364284, 33.777171], [-84.364284, 33.77701], [-84.364285, 33.776874], [-84.364286, 33.776746], [-84.364308, 33.776745], [-84.364464, 33.776744], [-84.364568, 33.776743], [-84.364568, 33.776743], [-84.364574, 33.776689], [-84.364742, 33.776687], [-84.365276, 33.776677], [-84.365289, 33.776677], [-84.365347, 33.776676], [-84.36515, 33.77587], [-84.364954, 33.775072], [-84.364863, 33.7747], [-84.364783, 33.774248], [-84.364696, 33.773752], [-84.364634, 33.773507], [-84.36462, 33.773451], [-84.364561, 33.773218], [-84.364412, 33.772632], [-84.364382, 33.772521], [-84.364281, 33.772143], [-84.364277, 33.772126], [-84.364229, 33.771991], [-84.364113, 33.77173], [-8